In [1]:
import os
import re
import pandas as pd
import io
import base64
from openai import OpenAI
import google.generativeai as genai
from collections import defaultdict
import pdfplumber # 이미지 추출을 위해 유지
import filetype
import camelot # 테이블 추출을 위해 추가
from unstructured.partition.pdf import partition_pdf

from pprint import pprint

In [ ]:
# --- 1. Gemini AI 클라이언트 및 유틸리티 함수 정의 ---

# 여기에 Gemini API 키를 설정하세요.
# 환경 변수에서 불러오는 것을 권장합니다.
# from google.colab import userdata
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)

genai.configure()

# 사용할 Gemini 모델 설정
# 요청하신 gemini-2.5pro는 아직 출시되지 않았으므로, gemini-1.5-pro-latest를 사용합니다.
# 추후 모델이 출시되면 이 부분만 수정하면 됩니다.
vision_model = genai.GenerativeModel('gemini-2.5-flash')
text_model = genai.GenerativeModel('gemini-2.5-flash')


def generate_image_description_with_gemini(image_path):
    """
    Gemini API를 사용하여 이미지 파일을 분석하고 구조화된 설명을 생성합니다.
    """
    print(f"\n--- 이미지 설명 생성 시작: {os.path.basename(image_path)} ---")
    try:
        img = Image.open(image_path)
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {image_path}")
        return None
    except Exception as e:
        print(f"오류: 이미지 파일 열기 실패 - {e}")
        return None

    prompt = f"""
    당신은 이미지를 분석하여 검색 시스템(RAG)을 위한 메타데이터를 생성하는 AI 전문가입니다.
    첨부된 이미지를 RAG 시스템에서 효과적으로 검색할 수 있도록, 아래 [분석 지침]에 따라 한글로 상세히 설명해주세요.

    [분석 지침]
    1.  **종합 요약 (1~2문장)**: 이미지의 핵심 주제와 내용을 간결하게 요약해주세요.
    2.  **주요 구성요소 및 객체**: 이미지에 포함된 중요한 사물, 인물, 아이콘, 그래프 요소 등을 구체적으로 나열해주세요.
    3.  **이미지 내 텍스트 (OCR)**: 이미지에 보이는 모든 텍스트를 그대로 옮겨 적어주세요. 텍스트가 없다면 '텍스트 없음'이라고 명시해주세요.
    4.  **시각적 특징 및 스타일**: 이미지의 전체적인 색상 톤, 구도, 스타일, 분위기 등을 설명해주세요.
    5.  **핵심 키워드 (쉼표로 구분)**: 검색에 사용될 만한 핵심 키워드를 5개 이상 나열해주세요.
    """

    try:
        response = vision_model.generate_content([prompt, img])
        return response.text
    except Exception as e:
        print(f"Gemini 이미지 분석 중 오류 발생: {e}")
        return "오류: 이미지 설명을 생성하지 못했습니다."


def generate_table_summary_with_gemini(table_csv_str):
    """
    Gemini API를 사용하여 CSV 문자열 데이터를 분석하고 요약 보고서를 생성합니다.
    """
    print(f"\n--- 테이블 요약 생성 시작 ---")

    prompt = f"""
    당신은 최고의 데이터 분석가입니다. 당신의 임무는 주어진 CSV 형식의 데이터를 구조적으로 분석하고, 비전문가도 이해하기 쉽게 핵심 내용을 요약하는 것입니다.

    아래 [분석 지침]과 [CSV 데이터]를 보고 상세한 분석 보고서를 한국어로 작성해 주세요.

    [분석 지침]
    1.  **표의 주제**: 이 표가 무엇에 대한 데이터인지 한 문장으로 명확하게 정의하세요.
    2.  **구조 설명**: 각 행(row)과 열(column)이 무엇을 나타내는지 설명하세요.
    3.  **핵심 정보 및 수치**: 표에서 가장 중요한 핵심 정보를 3~5가지 항목으로 요약하세요. 구체적인 수치, 비율(%), 조건, 기간 등을 반드시 포함하세요.
    4.  **패턴 또는 특이사항 (선택 사항)**: 데이터에서 발견할 수 있는 패턴, 경향성 또는 특이점이 있다면 언급하세요.
    5.  **참고**: 데이터는 행 또는 열들의 결합으로 되어있을 수도 있습니다. 행과 열 모두 신중히 보세요.

    [CSV 데이터]
    ---
    {table_csv_str}
    ---
    """

    try:
        response = text_model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Gemini 테이블 분석 중 오류 발생: {e}")
        return "오류: 테이블 요약을 생성하지 못했습니다."


def process_table_with_ai_camelot(table_df, output_dir, page_num, table_index):
    """camelot으로 추출한 테이블 DataFrame을 처리하고 Gemini AI 요약을 생성합니다."""
    try:
        table_filename = f"p{page_num}_tbl{table_index}.csv"
        table_path = os.path.join(output_dir, "tables", table_filename)
        table_df.to_csv(table_path, index=False, encoding="utf-8-sig")

        # AI 요약을 위해 DataFrame을 CSV 문자열로 변환
        csv_string = table_df.to_csv(index=False)
        table_description = generate_table_summary_with_gemini(csv_string)

        return {"path": table_path, "page": page_num, "dataframe": table_df, "description": table_description}
    except Exception as e:
        print(f"  - 테이블 처리 중 오류 발생: {e}")
        return None

def process_image_with_ai(image_data, output_dir, page_num, image_index):
    """pdfplumber로 추출한 이미지를 처리하고 Gemini AI 설명을 생성합니다."""
    try:
        image_bytes = image_data['stream'].get_data()
        kind = filetype.guess(image_bytes)
        image_ext = kind.extension if kind else "png"
        image_filename = f"p{page_num}_img{image_index}.{image_ext}"
        image_path = os.path.join(output_dir, "images", image_filename)

        with open(image_path, "wb") as img_file:
            img_file.write(image_bytes)

        image_description = generate_image_description_with_gemini(image_path)
        return {"path": image_path, "page": page_num, "description": image_description}
    except Exception as e:
        print(f"  - 이미지 처리 중 오류 발생: {e}")
        return None

# --- 2. camelot을 사용하도록 수정한 메인 함수 ---

def create_integrated_markdown_from_camelot(pdf_path):
    output_dir = os.path.basename(pdf_path).split(".")[0]
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "tables"), exist_ok=True)

    # 1단계: 미디어 정보 미리 추출
    print("1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...")

    # pdfplumber로 이미지 추출
    images_by_page = defaultdict(list)
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            images_by_page[page.page_number].extend(page.images)
    print(f"총 {sum(len(v) for v in images_by_page.values())}개의 이미지 정보를 찾았습니다.")

    # camelot으로 테이블 추출
    tables_by_page = defaultdict(list)
    try:
        tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', suppress_stdout=True)
        for table in tables:
            tables_by_page[table.page].append(table.df)
        print(f"총 {sum(len(v) for v in tables_by_page.values())}개의 테이블 정보를 찾았습니다.")
    except Exception as e:
        print(f"Camelot으로 테이블 추출 중 오류 발생: {e}")

    # 2단계: unstructured로 PDF 요소 순서대로 파티셔닝
    print("2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...")
    elements = partition_pdf(filename=pdf_path, infer_table_structure=True, strategy="hi_res")

    # 3단계: 요소 순회하며 최종 마크다운 및 데이터 생성
    print("3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...")
    final_markdown_parts = []
    extracted_tables = []
    extracted_images = []
    page_counters = defaultdict(lambda: {'tables': 0, 'images': 0})

    for el in elements:
        page_num = el.metadata.page_number

        if "unstructured.documents.elements.Table" in str(type(el)):
            table_index = page_counters[page_num]['tables']
            if table_index < len(tables_by_page[page_num]):
                raw_table_df = tables_by_page[page_num][table_index]
                table_data = process_table_with_ai_camelot(raw_table_df, output_dir, page_num, table_index)
                if table_data:
                    placeholder = f"\n\n[[-- TABLE: Page {page_num}, Index {table_index} | Path: {table_data['path']} --]]\n**표 요약:** {table_data['description']}\n\n"
                    final_markdown_parts.append(placeholder)
                    extracted_tables.append(table_data)
                    page_counters[page_num]['tables'] += 1

        elif "unstructured.documents.elements.Image" in str(type(el)):
            image_index = page_counters[page_num]['images']
            if image_index < len(images_by_page[page_num]):
                raw_image_data = images_by_page[page_num][image_index]
                image_data = process_image_with_ai(raw_image_data, output_dir, page_num, image_index)
                if image_data:
                    relative_path = os.path.relpath(image_data['path'], output_dir).replace(os.sep, '/')
                    md_link = f"\n\n![{image_data['description']}]({relative_path})\n\n"
                    final_markdown_parts.append(md_link)
                    extracted_images.append(image_data)
                    page_counters[page_num]['images'] += 1

        else:
            final_markdown_parts.append(el.text)

    # 4단계: 최종 결과 정리 및 저장
    print("4단계: 최종 결과 정리 및 저장...")
    final_markdown = "\n\n".join(final_markdown_parts)
    output_filename = os.path.join(output_dir, "integrated_markdown_gemini.md")
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(final_markdown)

    print("\n--- 작업 완료 ---")
    return {
        "output_dir": output_dir,
        "integrated_markdown_file": output_filename,
        "images": extracted_images,
        "tables": extracted_tables,
        "integrated_markdown_content": final_markdown,
    }

# --- 코드 실행 예제 ---
# 아래 코드 실행 전, 'YOUR_GOOGLE_API_KEY' 부분을 실제 키로 변경해야 합니다.
pdf_path = "datasets/manual.pdf"  # 실제 PDF 파일 경로로 변경해주세요.
extracted_data = create_integrated_markdown_from_camelot(pdf_path)

# # 결과 확인
# print(f"\nAI 요약/설명이 포함된 통합 마크다운 파일이 '{extracted_data['integrated_markdown_file']}'에 저장되었습니다.")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 0개의 이미지 정보를 찾았습니다.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 2개의 테이블 정보를 찾았습니다.
2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...

--- 테이블 요약 생성 시작 ---

--- 테이블 요약 생성 시작 ---
4단계: 최종 결과 정리 및 저장...

--- 작업 완료 ---


In [ ]:
# #gemini-2.5-flash

# from google import genai
# from google.genai import types
# import pathlib

# client = genai.Client()

# filepath = pathlib.Path("datasets/manual.pdf")

# prompt = """
#     RAG 시스템 구축을 위해 주어진 pdf를 마크다운 형식으로 만들어주세요.
#     pdf내의 사진이나 표가 있을 경우 사진이나 표에 대한 설명을 적어 주세요.
# """ 
# response = client.models.generate_content(
#     model="gemini-2.5-flash",
#     contents=[
#         types.Part.from_bytes(
#             data=filepath.read_bytes(),
#             mime_type="application/pdf",
#         ),
#         prompt
#     ]
# )

# print(response.text)

V 사업비 집행 및 정산지침

1 사업비 지급 및 집행기간
□ 사업비 지급

| 1~2분기             |                  | 3~4분기                        |             |
| :------------------- | :--------------- | :----------------------------- | :---------- |
| **1회차**            |                  | **2회차**                      |             |
| 신청 시기            | 지급 비율        | 신청 시기                      | 지급 비율   |
| 협약 체결 시         | 20%              | 협약 체결 후 2개월 이내        | 46%         |
| **3회차**            |                  | **4회차**                      |             |
| 신청 시기            | 지급 비율        | 신청 시기                      | 지급 비율   |
| 중간 평가 후 1개월 이내 | 9%               | 4분기 (10월)                   | 25%         |

*표: 사업비 지급 회차별 신청 시기 및 지급 비율을 나타내는 표*

※ 지급비율은 변동될 수 있으며, 중간평가 결과에 따라 3회차부터 지급을 유예할 수 있음

□ 사업비 집행기간
'24. 1. 1. ~ '24. 12. 31.
※ 동 기간 이외의 사용금액은 불인정 금액으로 간주하고 전액 반납해야 함
※ 단, 집행기간 연장에 대해서 사전 승인을 받을 시에는 그 기한까지는 인정

2 사업비 계좌관리 및 사용원칙
□ 사업비 계좌관리
○ (별도 계좌개설 필수) 각 지원 대상자는 당해 랩 운영지원금과 민간부담금을 관리하는 통장(법인명

In [ ]:
# # --- 1. AI 클라이언트 및 유틸리티 함수 정의 (기존과 동일) ---
# client = OpenAI() # API 키는 환경 변수에 설정 권장

# def encode_image_to_base64(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# def run_assistant_on_file(file_path, assistant_instructions, user_prompt_content):
#     """
#     파일을 업로드하고 지정된 지시에 따라 Assistant를 실행한 후 결과를 반환하는 범용 함수입니다.
#     """
#     print(f"1. '{os.path.basename(file_path)}' 파일 업로드 중...")
#     try:
#         file_object = client.files.create(
#             file=open(file_path, "rb"),
#             purpose="assistants"
#         )
#     except FileNotFoundError:
#         print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
#         return None
#     print(f"   - 파일 ID: {file_object.id}")
    
#     # 2. 어시스턴트 생성
#     print("2. 분석용 어시스턴트 생성 중...")
#     assistant = client.beta.assistants.create(
#         name="전문 분석가",
#         instructions=assistant_instructions,
#         model="gpt-4o",
#         tools=[{"type": "code_interpreter"}]
#     )
    
#     # 3. 대화를 위한 스레드 생성
#     print("3. 대화 스레드 생성 중...")
#     thread = client.beta.threads.create()
#     print(f"   - 스레드 ID: {thread.id}")

#     # 4. 스레드에 메시지 및 파일 추가
#     print("4. 분석 요청 메시지 및 파일 추가 중...")
#     client.beta.threads.messages.create(
#         thread_id=thread.id,
#         role="user",
#         content=user_prompt_content,
#         attachments=[{"file_id": file_object.id, "tools": [{"type": "code_interpreter"}]}]
#     )
    
#     # 5. 어시스턴트 실행
#     print("5. 어시스턴트 실행 및 분석 시작...")
#     run = client.beta.threads.runs.create(
#         thread_id=thread.id,
#         assistant_id=assistant.id
#     )

#     # 6. 실행 완료 대기
#     while run.status not in ["completed", "failed"]:
#         run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
#         print(f"   - 현재 상태: {run.status}")
#         time.sleep(3)

#     # 7. 결과 처리 및 리소스 정리
#     summary = None
#     if run.status == "completed":
#         print("6. 분석 완료. 결과 가져오는 중...")
#         messages = client.beta.threads.messages.list(thread_id=thread.id)
#         for message in messages.data:
#             if message.role == "assistant":
#                 summary = message.content[0].text.value
#                 break
#     else:
#         print(f"오류: 분석 실패 - {run.last_error}")

#     # 생성된 리소스 정리
#     print("7. 생성된 리소스 정리 중...")
#     try:
#         client.beta.assistants.delete(assistant.id)
#         client.files.delete(file_object.id)
#         client.beta.threads.delete(thread.id)
#         print("   - 리소스 정리 완료.")
#     except Exception as e:
#         print(f"   - 리소스 정리 중 오류 발생: {e}")

#     return summary


# def generate_image_description_with_assistant(image_path):
#     """
#     Assistant API를 사용하여 이미지 파일을 분석하고 구조화된 설명을 생성합니다.
#     """
#     print(f"\n--- 이미지 설명 생성 시작: {os.path.basename(image_path)} ---")
#     assistant_instructions = """
#     당신은 이미지를 분석하여 검색 시스템(RAG)을 위한 메타데이터를 생성하는 AI 전문가입니다.
#     주어진 이미지의 모든 시각적, 텍스트적 요소를 추출하여 구조화된 설명을 제공하는 임무를 맡았습니다.
#     """
    
#     user_prompt = f"""
#     첨부된 이미지를 RAG 시스템에서 효과적으로 검색할 수 있도록, 아래 [분석 지침]에 따라 한글로 상세히 설명해주세요.

#     [분석 지침]
#     1.  **종합 요약 (1~2문장)**: 이미지의 핵심 주제와 내용을 간결하게 요약해주세요.
#     2.  **주요 구성요소 및 객체**: 이미지에 포함된 중요한 사물, 인물, 아이콘, 그래프 요소 등을 구체적으로 나열해주세요.
#     3.  **이미지 내 텍스트 (OCR)**: 이미지에 보이는 모든 텍스트를 그대로 옮겨 적어주세요. 텍스트가 없다면 '텍스트 없음'이라고 명시해주세요.
#     4.  **시각적 특징 및 스타일**: 이미지의 전체적인 색상 톤, 구도, 스타일, 분위기 등을 설명해주세요.
#     5.  **핵심 키워드 (쉼표로 구분)**: 검색에 사용될 만한 핵심 키워드를 5개 이상 나열해주세요.
#     """
    
#     return run_assistant_on_file(image_path, assistant_instructions, user_prompt)

# def generate_table_summary_with_assistant(table_csv_str, temp_file_path="temp_table.csv"):
#     """
#     Assistant API를 사용하여 CSV 문자열 데이터를 분석하고 요약 보고서를 생성합니다.
#     """
#     print(f"\n--- 테이블 요약 생성 시작 ---")
    
#     # CSV 문자열을 임시 파일로 저장
#     try:
#         with open(temp_file_path, 'w', encoding='utf-8') as f:
#             f.write(table_csv_str)
#     except Exception as e:
#         print(f"오류: 임시 CSV 파일 생성 실패 - {e}")
#         return None

#     assistant_instructions = """
#     당신은 최고의 데이터 분석가입니다. 당신의 임무는 CSV 형식의 데이터를 구조적으로 분석하고, 비전문가도 이해하기 쉽게 핵심 내용을 요약하는 것입니다.
#     항상 다음 분석 절차를 엄격히 준수하여 답변을 생성해 주세요.
#     """
    
#     user_prompt = f"""
#     첨부된 CSV 데이터를 보고 아래 [분석 지침]에 따라 상세한 분석 보고서를 작성해 주세요.

#     [분석 지침]
#     1.  **표의 주제**: 이 표가 무엇에 대한 데이터인지 한 문장으로 명확하게 정의하세요.
#     2.  **구조 설명**: 각 행(row)과 열(column)이 무엇을 나타내는지 설명하세요.
#     3.  **핵심 정보 및 수치**: 표에서 가장 중요한 핵심 정보를 3~5가지 항목으로 요약하세요. 구체적인 수치, 비율(%), 조건, 기간 등을 반드시 포함하세요.
#     4.  **패턴 또는 특이사항 (선택 사항)**: 데이터에서 발견할 수 있는 패턴, 경향성 또는 특이점이 있다면 언급하세요.
#     5.  **참고**: 데이터는 행 또는 열들의 결합으로 되어있을 수도 있습니다. 행과 열 모두 신중히 보세요.
#     """
    
#     # user_prompt = """
#     # "첨부된 CSV 파일을 분석해서 다음 항목에 따라 보고서를 작성해 주세요:
#     # \n1. 표의 주제\n2. 구조(행, 열) 설명\n3. 핵심 정보 및 수치 요약\n4. 데이터 패턴 또는 특이사항\n\n이 보고서는 한국어로 작성되어야 합니다.",
#     # """
    
#     summary = run_assistant_on_file(temp_file_path, assistant_instructions, user_prompt)
    
#     # 임시 파일 삭제
#     if os.path.exists(temp_file_path):
#         os.remove(temp_file_path)
        
#     return summary


# def process_table_with_ai_camelot(table_df, output_dir, page_num, table_index):
#     """camelot으로 추출한 테이블 DataFrame을 처리하고 AI 요약을 생성합니다."""
#     try:
#         df = table_df
#         table_filename = f"p{page_num}_tbl{table_index}.csv"
#         table_path = os.path.join(output_dir, "tables", table_filename)
#         df.to_csv(table_path, index=False, encoding="utf-8-sig")
        
#         # AI 요약을 위해 DataFrame을 CSV 문자열로 변환
#         csv_string = df.to_csv(index=False)
#         table_description = generate_table_summary_with_assistant(csv_string)
        
#         return {"path": table_path, "page": page_num, "dataframe": df, "description": table_description}
#     except Exception as e:
#         print(f"  - 테이블 처리 중 오류 발생: {e}")
#         return None

# # process_image_with_ai 함수는 기존과 동일하게 pdfplumber를 사용하므로 변경 없음
# def process_image_with_ai(image_data, output_dir, page_num, image_index):
#     try:
#         image_bytes = image_data['stream'].get_data()
#         kind = filetype.guess(image_bytes)
#         image_ext = kind.extension if kind else "png"
#         image_filename = f"p{page_num}_img{image_index}.{image_ext}"
#         image_path = os.path.join(output_dir, "images", image_filename)
#         with open(image_path, "wb") as img_file:
#             img_file.write(image_bytes)
#         image_description = generate_image_description_with_assistant(image_path)
#         return {"path": image_path, "page": page_num, "description": image_description}
#     except Exception as e:
#         print(f"  - 이미지 처리 중 오류 발생: {e}")
#         return None

# # --- 3. camelot을 사용하도록 수정한 메인 함수 ---

# def create_integrated_markdown_from_camelot(pdf_path):
#     output_dir = os.path.basename(pdf_path).split(".")[0]
#     os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
#     os.makedirs(os.path.join(output_dir, "tables"), exist_ok=True)

#     # 1단계: pdfplumber로 이미지, camelot으로 테이블 미리 추출
#     print("1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...")
    
#     # pdfplumber로 이미지 추출
#     images_by_page = defaultdict(list)
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             images_by_page[page.page_number].extend(page.images)
#     print(f"총 {sum(len(v) for v in images_by_page.values())}개의 이미지 정보를 찾았습니다.")

#     # camelot으로 테이블 추출
#     # flavor='lattice'는 선이 있는 표에 적합, 선이 없는 표는 'stream' 사용
#     tables_by_page = defaultdict(list)
#     try:
#         tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', suppress_stdout=True)
#         for table in tables:
#             tables_by_page[table.page].append(table.df)
#         print(f"총 {sum(len(v) for v in tables_by_page.values())}개의 테이블 정보를 찾았습니다.")
#     except Exception as e:
#         print(f"Camelot으로 테이블 추출 중 오류 발생: {e}")

#     # 2단계: unstructured로 PDF 요소 순서대로 파티셔닝 (기존과 동일)
#     print("2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...")
#     elements = partition_pdf(filename=pdf_path, infer_table_structure=True, strategy="hi_res")

#     # 3단계: 요소 순회하며 최종 마크다운 및 데이터 생성
#     print("3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...")
#     final_markdown_parts = []
#     extracted_tables = []
#     extracted_images = []
#     page_counters = defaultdict(lambda: {'tables': 0, 'images': 0})

#     for el in elements:
#         page_num = el.metadata.page_number
        
#         if "unstructured.documents.elements.Table" in str(type(el)):
#             table_index = page_counters[page_num]['tables']
#             if table_index < len(tables_by_page[page_num]):
#                 # camelot이 추출한 DataFrame을 처리
#                 raw_table_df = tables_by_page[page_num][table_index]
#                 table_data = process_table_with_ai_camelot(raw_table_df, output_dir, page_num, table_index)
#                 if table_data:
#                     placeholder = f"\n\n[[-- TABLE: Page {page_num}, Index {table_index} | Path: {table_data['path']} --]]\n**표 요약:** {table_data['description']}\n\n"
#                     final_markdown_parts.append(placeholder)
#                     extracted_tables.append(table_data)
#                     page_counters[page_num]['tables'] += 1
            
#         elif "unstructured.documents.elements.Image" in str(type(el)):
#             image_index = page_counters[page_num]['images']
#             if image_index < len(images_by_page[page_num]):
#                 # pdfplumber가 추출한 이미지 처리
#                 raw_image_data = images_by_page[page_num][image_index]
#                 image_data = process_image_with_ai(raw_image_data, output_dir, page_num, image_index)
#                 if image_data:
#                     relative_path = os.path.relpath(image_data['path'], output_dir).replace(os.sep, '/')
#                     md_link = f"\n\n![{image_data['description']}]({relative_path})\n\n"
#                     final_markdown_parts.append(md_link)
#                     extracted_images.append(image_data)
#                     page_counters[page_num]['images'] += 1
        
#         else:
#             final_markdown_parts.append(el.text)

#     # 4단계: 최종 결과 정리 및 저장 (기존과 동일)
#     print("4단계: 최종 결과 정리 및 저장...")
#     final_markdown = "\n\n".join(final_markdown_parts)
#     output_filename = os.path.join(output_dir, "integrated_markdown_camelot.md")
#     with open(output_filename, "w", encoding="utf-8") as f:
#         f.write(final_markdown)
    
#     print("\n--- 작업 완료 ---")
#     return {
#         "output_dir": output_dir,
#         "integrated_markdown_file": output_filename,
#         "images": extracted_images,
#         "tables": extracted_tables,
#         "integrated_markdown_content": final_markdown,
#     }

# # --- 코드 실행 예제 ---
# pdf_path = "datasets/manual.pdf" # 실제 PDF 파일 경로로 변경해주세요.
# extracted_data = create_integrated_markdown_from_camelot(pdf_path)

# # # 결과 확인
# # print(f"\nAI 요약/설명이 포함된 통합 마크다운 파일이 '{extracted_data['integrated_markdown_file']}'에 저장되었습니다.")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 0개의 이미지 정보를 찾았습니다.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 2개의 테이블 정보를 찾았습니다.
2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...

--- 테이블 요약 생성 시작 ---
1. 'temp_table.csv' 파일 업로드 중...
   - 파일 ID: file-AetHJgnrsTitC8hZVGQDnn
2. 분석용 어시스턴트 생성 중...
  - 테이블 처리 중 오류 발생: Error code: 400 - {'error': {'message': "The requested model 'o3-mini' cannot be used with the 'code_interpreter' tool.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'unsupported_model'}}

--- 테이블 요약 생성 시작 ---
1. 'temp_table.csv' 파일 업로드 중...
   - 파일 ID: file-4598CnS6fcgTnjV7WGHwFq
2. 분석용 어시스턴트 생성 중...
  - 테이블 처리 중 오류 발생: Error code: 400 - {'error': {'message': "The requested model 'o3-mini' cannot be used with the 'code_interpreter' tool.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'unsupported_model'}}
4단계: 최종 결과 정리 및 저장...

--- 작업 완료 ---


In [2]:
import torch
import re
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document  # Document 클래스를 import 합니다.

# --- 1. 메타데이터를 포함하여 청킹하고 Document 객체를 생성하는 함수 ---

def create_documents_from_markdown(markdown_content, source_path):
    """
    논리적 청킹을 수행하고, 각 청크를 메타데이터가 포함된 LangChain Document 객체로 변환합니다.

    Args:
        markdown_content (str): 청킹할 마크다운 전체 텍스트
        source_path (str): 원본 파일 경로 (메타데이터에 사용)

    Returns:
        list[Document]: 내용과 메타데이터가 포함된 Document 객체 리스트
    """
    # 기존의 논리적 청킹 함수를 그대로 사용합니다.
    text_chunks = chunk_markdown_logically(markdown_content)
    
    documents = []
    for chunk_text in text_chunks:
        metadata = {"source": source_path}  # 모든 청크에 기본 출처 정보 추가
        
        # 청크 내용을 기반으로 추가 메타데이터 파싱
        if chunk_text.startswith("[[-- TABLE:"):
            metadata['type'] = 'table'
            # 정규표현식을 사용하여 페이지 번호와 경로 추출
            page_match = re.search(r"Page (\d+)", chunk_text)
            path_match = re.search(r"Path: (.*?)\s*--]]", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if path_match:
                metadata['table_path'] = path_match.group(1).strip()

        elif chunk_text.startswith("!["):
            metadata['type'] = 'image'
            # 정규표현식을 사용하여 페이지 번호와 이미지 설명 추출
            page_match = re.search(r"Image from page (\d+)", chunk_text)
            desc_match = re.search(r"!\[(.*?)\]\(", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if desc_match:
                metadata['description'] = desc_match.group(1).strip()
        else:
            metadata['type'] = 'text'

        # Document 객체 생성
        doc = Document(page_content=chunk_text, metadata=metadata)
        documents.append(doc)
        
    return documents

# --- 2. 기존 로직 실행 및 all_chunks 생성 ---

# chunk_markdown_logically 함수는 제공된 코드에 이미 정의되어 있다고 가정합니다.
def chunk_markdown_logically(markdown_content):
    """
    문서의 구조(제목, 표, 이미지)를 이해하여 논리적인 단위로 청킹합니다.
    RAG 시스템에 가장 효과적인 방법입니다.
    """
    # 1. 기본 블록으로 분리
    blocks = markdown_content.split('\n\n')
    
    chunks = []
    current_chunk = ""

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        # 2. 규칙에 따라 청크 생성
        is_table_placeholder = block.startswith("[[-- TABLE:")
        is_image_link = block.startswith("![")
        # 제목으로 사용될 수 있는 패턴들 (필요에 따라 추가)
        is_heading = block.startswith(('□', '○', '※')) or re.match(r'^[0-9]+\s|^\w+\s', block) and len(block) < 50

        if is_table_placeholder or is_image_link:
            # 테이블/이미지는 독립적인 청크로 처리
            if current_chunk:
                chunks.append(current_chunk.strip())
            chunks.append(block)
            current_chunk = ""
        elif is_heading:
            # 제목은 다음 블록과 합치기 위해, 진행 중인 청크를 먼저 저장
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = block
        else:
            # 일반 텍스트는 현재 청크(제목 또는 이전 텍스트)에 추가
            if current_chunk:
                current_chunk += "\n\n" + block
            else:
                current_chunk = block
    
    # 마지막 남은 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

source_file_path = "manual/integrated_markdown_gemini.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 'all_chunks' 변수는 이제 단순 문자열 리스트가 아닌,
# 내용과 메타데이터를 모두 포함한 Document 객체의 리스트가 됩니다.
all_chunks = create_documents_from_markdown(content, source_file_path)

pprint(all_chunks)

[Document(metadata={'source': 'manual/integrated_markdown_gemini.md', 'type': 'text'}, page_content='Ⅴ 사업비 집행 및 정산지침\n\nOE'),
 Document(metadata={'source': 'manual/integrated_markdown_gemini.md', 'type': 'text'}, page_content='사업비 지급 및 집행기간'),
 Document(metadata={'source': 'manual/integrated_markdown_gemini.md', 'type': 'text'}, page_content='□ 사업비 지급'),
 Document(metadata={'source': 'manual/integrated_markdown_gemini.md', 'type': 'table', 'page': 1, 'table_path': 'manual\\tables\\p1_tbl0.csv'}, page_content='[[-- TABLE: Page 1, Index 0 | Path: manual\\tables\\p1_tbl0.csv --]]\n**표 요약:** 최고의 데이터 분석가로서, 주어진 CSV 데이터를 분석하여 비전문가도 쉽게 이해할 수 있도록 핵심 내용을 구조적으로 요약한 보고서를 작성합니다.'),
 Document(metadata={'source': 'manual/integrated_markdown_gemini.md', 'type': 'text'}, page_content="---\n\n### **데이터 분석 보고서**\n\n**1. 표의 주제**\n\n이 표는 특정 사업 또는 프로젝트에 대한 **분기별, 회차별 지급 비율 및 관련 신청 시기**를 나타냅니다.\n\n**2. 구조 설명**\n\n이 표는 여러 겹의 헤더 구조를 가지고 있습니다.\n*   **가장 상단 행 (1~2분기, 3~4분기):** 전체 기간을 크게 **'1~2분기'**와 **'3~4분기'**

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_file_path = "manual/integrated_markdown_camelot.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    file_content = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100, 
    length_function=len,
)

# .split_text() 메서드를 사용하여 텍스트를 분할합니다.
split_texts = text_splitter.split_text(file_content)
pprint(split_texts)

['Ⅴ 사업비 집행 및 정산지침\n'
 '\n'
 'OE\n'
 '\n'
 '사업비 지급 및 집행기간\n'
 '\n'
 '□ 사업비 지급\n'
 '\n'
 '\n'
 '\n'
 '[[-- TABLE: Page 1, Index 0 | Path: manual\\tables\\p1_tbl0.csv --]]\n'
 '**표 요약:** 주어진 데이터를 분석해 보겠습니다.\n'
 '\n'
 '### 1. 표의 주제\n'
 '이 표는 특정 시점에 대한 지급 비율을 표시하고 있는 데이터입니다. 구체적으로는 각 분기의 특정 시기에 따라 지급 비율이 어떻게 '
 '변하는지를 나타냅니다.\n'
 '\n'
 '### 2. 구조 설명\n'
 '- **행**:\n'
 '  - 0번 행: 분기를 정의하는 행으로, 데이터가 어떤 시기(예: 1~2분기, 3~4분기)에 관한 것인지 표시합니다.\n'
 '  - 1번 행: 회차를 정의하는 행으로, 각 분기에 대한 상세한 회차 정보를 보여줍니다(1회차, 2회차 등).\n'
 "  - 2번 행: 데이터 컬럼 헤더에 해당하며, '신청 시기'와 '지급 비율'이 반복됩니다.\n"
 '  - 3번 행: 각 회차별로 신청 시기와 그에 따른 지급 비율을 구체적으로 나타냅니다.',
 '- **열**:\n'
 '  - 각 열은 신청 시기와 그에 해당하는 지급 비율로 이루어져 있습니다.\n'
 '\n'
 '### 3. 핵심 정보 및 수치\n'
 '- **1~2분기**:\n'
 '  - **1회차**: 협약 체결 시 20% 지급.\n'
 '  - **2회차**: 협약 체결 후 2개월 이내 46% 지급.\n'
 '\n'
 '- **3~4분기**:\n'
 '  - **3회차**: 중간 평가 후 1개월 이내 9% 지급.\n'
 '  - **4회차**: 4분기(10월) 25% 지급.\n'
 '\n'
 '### 4. 패턴 또는 특이사항\n'
 '- 각 분기의 초기에는 높은 비율의 지급이 이루어지고, 시간이 흐름에 따라 지급 비율이 감소하는 패턴을 보입니다. 이로 인해 초기에

In [4]:
# --- 3. Vector Store 생성 및 저장 ---

print("\n--- Vector Store 생성 시작 ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"임베딩 모델을 위한 디바이스 설정: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# Chroma.from_documents 함수는 Document 객체 리스트를 받도록 설계되어 있습니다.
vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # DB를 저장할 디렉토리
)

print("\n--- Vector Store 생성 및 저장 완료 ---")
print(f"'{vectorstore._persist_directory}' 디렉토리에 {len(all_chunks)}개의 청크가 저장되었습니다.")


--- Vector Store 생성 시작 ---
임베딩 모델을 위한 디바이스 설정: cuda

--- Vector Store 생성 및 저장 완료 ---
'./chroma_db' 디렉토리에 41개의 청크가 저장되었습니다.


In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever()


prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
# llm = OllamaLLM(model="gemma3:4b")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
question = "2회차 지급 비율과 지급 시기는?"
# 테이블 정보를 최대한 정밀하게 추출할 수 있도록 하기
answer = chain.invoke(question)
print(answer)

2회차 지급 비율은 협약 체결 후 2개월 이내에 46%이고, 지급 시기는 협약 체결 후 2개월 이내입니다.
